In [1]:
import pandas as pd

### 데이터 베이스 설정

In [2]:
DB_URL = 'onad.cbjjamtlar2t.ap-northeast-2.rds.amazonaws.com'
DB_USER = 'onad'
DB_PASSWORD = 'rkdghktn12'
DB_DATABASE = 'onad'
DB_CHARSET = 'utf8mb4'
DB_LOGFLAG  = 'False'
DB_PORT = 3306

db_url = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    DB_USER, DB_PASSWORD,
    DB_URL, DB_PORT,
    DB_DATABASE, DB_CHARSET
)

In [3]:
def data_from_sql_twitchchat(streamer, target_date):
    query = """
        select * from twitch_chat where streamer_name = "%s" and broad_date ="%s"
    """% (streamer, target_date)

    return pd.read_sql_query(query, db_url)

### 채팅 데이터 로드

In [105]:
streamer = "looksam"
target_date = "2018-12-27"
chat_data = data_from_sql_twitchchat(streamer, target_date)

### Exploratory Data Analysis

In [106]:
chat_data.shape

(64193, 6)

In [107]:
chat_data.head()

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents
0,5235567,looksam,2018-12-27,<$Lee_sang_hyuk>,17:43:26,오늘방송이 안켜진다면
1,5235568,looksam,2018-12-27,<$Lee_sang_hyuk>,17:43:36,아마도 뜨겁겠지..?
2,5235569,looksam,2018-12-27,<rnwhdan>,17:43:45,오뱅없?
3,5235570,looksam,2018-12-27,<%당근빠따 (ambus000)>,18:17:10,looksamJun looksamJun looksamJun
4,5235571,looksam,2018-12-27,<rnwhdan>,18:31:22,절 대 방 송 켜


In [108]:
print("채팅 로그의 수: %s" % (chat_data.shape[0]))
print("순수 시청자 수: %s" % (len(list(chat_data['chatterer'].unique()))))

채팅 로그의 수: 64193
순수 시청자 수: 4415


In [109]:
print("채팅로그의 시작과 끝 : %s ~ %s " % (sorted(chat_data['chat_time'])[0], sorted(chat_data['chat_time'])[-1]))

채팅로그의 시작과 끝 : 17:43:26 ~ 23:59:59 


### 시간데이터를 DatetimeIndex 로 변경

In [110]:
chat_data['time'] = pd.to_datetime(
    chat_data['broad_date'].apply(lambda x : ",".join(x.split("-"))) + "," + \
    chat_data['chat_time'].apply(lambda x : ",".join(x.split(":"))), 
                                  format="%Y,%m,%d,%H,%M,%S")


In [111]:
chat_data_reindex = chat_data.set_index('time')

In [112]:
chat_data_reindex.head(2)

,chat_id,streamer_name,broad_date,chatterer,chat_time,chat_contents
time,,,,,,
2018-12-27 17:43:26,5235567,looksam,2018-12-27,<$Lee_sang_hyuk>,17:43:26,오늘방송이 안켜진다면
2018-12-27 17:43:36,5235568,looksam,2018-12-27,<$Lee_sang_hyuk>,17:43:36,아마도 뜨겁겠지..?


### pd.Datetime index 생성

In [113]:
# YYYY-mm-dd HH:MM:SS 의 datetime 객체 인덱스 생성 (00:00:00 ~ 23:59:59 까지)
index=pd.date_range(target_date, periods=86400, freq="S")
# 인덱스만 가지는 데이터 프레임 생성 (chat_data와 병합하기 위함)
df = pd.DataFrame(index=index,)

In [114]:
# 필요없는 행 삭제
del chat_data_reindex['chat_id']
del chat_data_reindex['streamer_name']
del chat_data_reindex['broad_date']
del chat_data_reindex['chat_time']

In [115]:
chat_data_reindex.head(2)

,chatterer,chat_contents
time,,
2018-12-27 17:43:26,<$Lee_sang_hyuk>,오늘방송이 안켜진다면
2018-12-27 17:43:36,<$Lee_sang_hyuk>,아마도 뜨겁겠지..?


### 한 문장에서 특정 단어의 수 세어 컬럼으로 생성

In [116]:
chat_data_reindex['cnt_ㅋ'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("ㅋ"))
chat_data_reindex['cnt_ㄵ'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("ㄵ"))
chat_data_reindex['cnt_ㄴㅈ'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("ㄴㅈ"))
chat_data_reindex['cnt_오'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("오"))
chat_data_reindex['cnt_와'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("와"))
chat_data_reindex['cnt_유하'] = chat_data_reindex.chat_contents.apply(lambda x : x.count("유하"))


In [117]:
chat_data_reindex.head()

,chatterer,chat_contents,cnt_ㅋ,cnt_ㄵ,cnt_ㄴㅈ,cnt_오,cnt_와,cnt_유하
time,,,,,,,,
2018-12-27 17:43:26,<$Lee_sang_hyuk>,오늘방송이 안켜진다면,0,0,0,1,0,0
2018-12-27 17:43:36,<$Lee_sang_hyuk>,아마도 뜨겁겠지..?,0,0,0,0,0,0
2018-12-27 17:43:45,<rnwhdan>,오뱅없?,0,0,0,1,0,0
2018-12-27 18:17:10,<%당근빠따 (ambus000)>,looksamJun looksamJun looksamJun,0,0,0,0,0,0
2018-12-27 18:31:22,<rnwhdan>,절 대 방 송 켜,0,0,0,0,0,0


### 해당 채팅 수 빈도를 갖는 데이터 프레임 생성

In [129]:
pivot_df = chat_data_reindex.pivot_table(index=chat_data_reindex.index, aggfunc=len, values='chatterer')
pivot_df.columns = ['cnt_chat']  # 컬럼 이름 할당
pivot_df["cnt_ㅋ"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_ㅋ')['cnt_ㅋ']
pivot_df["cnt_ㄵ"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_ㄵ')['cnt_ㄵ'] +\
                    chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_ㄴㅈ')['cnt_ㄴㅈ']
pivot_df["cnt_오"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_오')['cnt_오']
pivot_df["cnt_와"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_와')['cnt_와']
pivot_df["cnt_유하"] = chat_data_reindex.pivot_table(index = chat_data_reindex.index, aggfunc=sum, values='cnt_유하')['cnt_유하']


# 초당 채팅수가 평균 이상인 경우만 보이도록
pivot_df[pivot_df['cnt_chat'] > pivot_df['cnt_chat'].mean()]

print(sorted(pivot_df['cnt_chat'].unique(), reverse=True)[:10])
print(sorted(pivot_df['cnt_ㅋ'].unique(), reverse=True)[:10])
print(sorted(pivot_df['cnt_유하'].unique(), reverse=True))

pivot_df[pivot_df['cnt_유하'] == 5]  # 해당 글자가 가장 높은 구간

[39, 38, 36, 33, 32, 31, 28, 27, 26, 25]
[371, 334, 316, 302, 296, 267, 264, 251, 248, 238]
[5, 3, 2, 1, 0]


,cnt_chat,cnt_ㅋ,cnt_ㄵ,cnt_오,cnt_와,cnt_유하
time,,,,,,
2018-12-27 20:19:54,13,0,0,0,0,5


### 해당 글자의 빈도수가 높은 구간의 채팅

In [130]:
chat_data_reindex[chat_data_reindex.index == "2018-12-27 20:19:54"]

,chatterer,chat_contents,cnt_ㅋ,cnt_ㄵ,cnt_ㄴㅈ,cnt_오,cnt_와,cnt_유하
time,,,,,,,,
2018-12-27 20:19:54,<야지로베 (bestofbestofbestofbest)>,?,0,0,0,0,0,0
2018-12-27 20:19:54,<ace02ahn>,유하,0,0,0,0,0,1
2018-12-27 20:19:54,<클린노루 (u090)>,루 밍,0,0,0,0,0,0
2018-12-27 20:19:54,<%gurakim>,?,0,0,0,0,0,0
2018-12-27 20:19:54,<intrigue1997>,ㅇㄱㅇ,0,0,0,0,0,0
2018-12-27 20:19:54,<냠냠냠냠2 (yaaaaaaaaap)>,QNA??,0,0,0,0,0,0
2018-12-27 20:19:54,<으악새는으악울지 (euaksae)>,유하,0,0,0,0,0,1
2018-12-27 20:19:54,<%브니 (vnivni)>,유하,0,0,0,0,0,1
2018-12-27 20:19:54,<dogsell2>,수금타임,0,0,0,0,0,0


### 시작시간 불러와 시작시간으로부터 00:00:00 으로 시작하는 컬럼만들기

In [141]:
streamer_id = 103861159
query = "select * from twitch_stream where streamer_id = '%s'" % streamer_id
broad_date_df = pd.read_sql_query(query, db_url)

broad = pd.to_datetime(broad_date_df['broad_date'])
broad.apply(lambda x : x + pd.Timedelta("9hours"))  # 트위치 시간대는 UTC 기준이므로 한국시간으로 +9 하여 변경

0   2019-01-05 18:09:46
1   2019-01-07 18:04:45
Name: broad_date, dtype: datetime64[ns]

### 인덱스를 00 부터 시작

In [294]:
# 데이터프레임 복사
index_new_df = chat_data_reindex.copy()
index_new_df.index[0]

Timestamp('2018-12-27 17:43:26')

In [295]:
# 방송시작시간이 다음과 같다면
start_time = "2018-12-27 18:53:50"
start_time = pd.to_datetime(start_time)


In [296]:
# 첫 채팅시간
first_chat_time = chat_data_reindex[chat_data_reindex.index == "2018-12-27 17:43:26"].index.unique()
first_chat_time

DatetimeIndex(['2018-12-27 17:43:26'], dtype='datetime64[ns]', name='time', freq=None)

In [297]:
# 첫 채팅 시간 - 방송 시작 시간 을 통해 방송시작시간으로부터 몇초가 지나고 있는지 확인 가능
first_chat_time - start_time

TimedeltaIndex(['-1 days +22:49:36'], dtype='timedelta64[ns]', name='time', freq=None)

In [298]:
# 첫 채팅 시간 - 방송 시작시간을 인덱스로
index_new_df['korea_time'] = index_new_df.index
index_new_df.index = index_new_df['korea_time'].apply(lambda x : x - start_time)
index_new_df.index.name = "stream_time"

In [315]:
# 인덱스 중 방송 시작 이전의 데이터는 제외한 나머지
new_df = index_new_df[index_new_df.index >= "00:00:00"]

### 인덱스를 기준으로 피봇팅

In [324]:
# 채팅 빈도수 데이터 준비
new_pivot_df1 = new_df.pivot_table(index=new_df.index, values=['chat_contents'], aggfunc=len)
# 이외의 빈도수 데이터 준비
new_pivot_df2 = new_df.pivot_table(index=new_df.index,
                                   values=['cnt_ㅋ','cnt_ㄵ','cnt_ㄴㅈ','cnt_오', 'cnt_와', 'cnt_유하'],
                                  aggfunc=sum)

In [359]:
anal_df = new_pivot_df1.merge(new_pivot_df2, on=new_pivot_df1.index)
anal_df.set_index('key_0', inplace=True)
anal_df.index.name = "stream_time"

In [360]:
anal_df.head()

,chat_contents,cnt_ㄴㅈ,cnt_ㄵ,cnt_ㅋ,cnt_오,cnt_와,cnt_유하
stream_time,,,,,,,
00:00:02,1,0,0,3,0,1,0
00:00:03,1,0,0,0,0,0,0
00:00:04,6,0,0,1,3,0,0
00:00:05,1,0,0,0,0,0,0
00:00:06,1,0,0,0,0,0,0
